for details, refer to https://www.kaggle.com/competitions/leash-BELKA/discussion/498858

In [2]:
# !pip install rdkit
# !pip install torch_geometric
# !pip install torch-scatter

In [1]:
import numpy as np

import rdkit
from rdkit import Chem
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import pandas as pd
from torch_geometric.nn import MessagePassing, global_mean_pool
from torch_scatter import scatter
import polars as pl
import torch
import torch.nn as nn
import torch.nn.functional as F

print('import ok!')

import ok!


In [78]:


class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = True
    
    SEED = 42
    EPOCHS = 4
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 1e-6*0
    PATIENCE = 10
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]
    EARLY_STOPPING = False
    
    
if Config.DEBUG:
    n_rows = 10**4*3
else:
    n_rows = None
    

In [79]:
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"

TRAIN_DATA_NAME = "train_enc.parquet"

In [80]:
# helper
# torch version of np unpackbits
#https://gist.github.com/vadimkantorov/30ea6d278bc492abf6ad328c6965613a

def tensor_dim_slice(tensor, dim, dim_slice):
	return tensor[(dim if dim >= 0 else dim + tensor.dim()) * (slice(None),) + (dim_slice,)]

# @torch.jit.script
def packshape(shape, dim: int = -1, mask: int = 0b00000001, dtype=torch.uint8, pack=True):
	dim = dim if dim >= 0 else dim + len(shape)
	bits, nibble = (
		8 if dtype is torch.uint8 else 16 if dtype is torch.int16 else 32 if dtype is torch.int32 else 64 if dtype is torch.int64 else 0), (
		1 if mask == 0b00000001 else 2 if mask == 0b00000011 else 4 if mask == 0b00001111 else 8 if mask == 0b11111111 else 0)
	# bits = torch.iinfo(dtype).bits # does not JIT compile
	assert nibble <= bits and bits % nibble == 0
	nibbles = bits // nibble
	shape = (shape[:dim] + (int(math.ceil(shape[dim] / nibbles)),) + shape[1 + dim:]) if pack else (
				shape[:dim] + (shape[dim] * nibbles,) + shape[1 + dim:])
	return shape, nibbles, nibble

# @torch.jit.script
def F_unpackbits(tensor, dim: int = -1, mask: int = 0b00000001, shape=None, out=None, dtype=torch.uint8):
	dim = dim if dim >= 0 else dim + tensor.dim()
	shape_, nibbles, nibble = packshape(tensor.shape, dim=dim, mask=mask, dtype=tensor.dtype, pack=False)
	shape = shape if shape is not None else shape_
	out = out if out is not None else torch.empty(shape, device=tensor.device, dtype=dtype)
	assert out.shape == shape

	if shape[dim] % nibbles == 0:
		shift = torch.arange((nibbles - 1) * nibble, -1, -nibble, dtype=torch.uint8, device=tensor.device)
		shift = shift.view(nibbles, *((1,) * (tensor.dim() - dim - 1)))
		return torch.bitwise_and((tensor.unsqueeze(1 + dim) >> shift).view_as(out), mask, out=out)

	else:
		for i in range(nibbles):
			shift = nibble * i
			sliced_output = tensor_dim_slice(out, dim, slice(i, None, nibbles))
			sliced_input = tensor.narrow(dim, 0, sliced_output.shape[dim])
			torch.bitwise_and(sliced_input >> shift, mask, out=sliced_output)
	return out

class dotdict(dict):
	__setattr__ = dict.__setitem__
	__delattr__ = dict.__delitem__
	
	def __getattr__(self, name):
		try:
			return self[name]
		except KeyError:
			raise AttributeError(name)

            
print('helper ok!')

helper ok!


In [81]:
# mol to graph adopted from
# from https://github.com/LiZhang30/GPCNDTA/blob/main/utils/DrugGraph.py

PACK_NODE_DIM=9
PACK_EDGE_DIM=1
NODE_DIM=PACK_NODE_DIM*8
EDGE_DIM=PACK_EDGE_DIM*8

def one_of_k_encoding(x, allowable_set, allow_unk=False):
	if x not in allowable_set:
		if allow_unk:
			x = allowable_set[-1]
		else:
			raise Exception(f'input {x} not in allowable set{allowable_set}!!!')
	return list(map(lambda s: x == s, allowable_set))


#Get features of an atom (one-hot encoding:)
'''
	1.atom element: 44+1 dimensions    
	2.the atom's hybridization: 5 dimensions
	3.degree of atom: 6 dimensions                        
	4.total number of H bound to atom: 6 dimensions
	5.number of implicit H bound to atom: 6 dimensions    
	6.whether the atom is on ring: 1 dimension
	7.whether the atom is aromatic: 1 dimension           
	Total: 70 dimensions
'''

ATOM_SYMBOL = [
	'C', 'N', 'O', 'S', 'F', 'Si', 'P', 'Cl', 'Br', 'Mg',
	'Na', 'Ca', 'Fe', 'As', 'Al', 'I', 'B', 'V', 'K', 'Tl',
	'Yb', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn', 'H',
	'Li', 'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'In', 'Mn', 'Zr', 'Cr',
	'Pt', 'Hg', 'Pb', 'Dy',
	#'Unknown'
]
#print('ATOM_SYMBOL', len(ATOM_SYMBOL))44
HYBRIDIZATION_TYPE = [
	Chem.rdchem.HybridizationType.S,
	Chem.rdchem.HybridizationType.SP,
	Chem.rdchem.HybridizationType.SP2,
	Chem.rdchem.HybridizationType.SP3,
	Chem.rdchem.HybridizationType.SP3D
]

def get_atom_feature(atom):
	feature = (
		 one_of_k_encoding(atom.GetSymbol(), ATOM_SYMBOL)
	   + one_of_k_encoding(atom.GetHybridization(), HYBRIDIZATION_TYPE)
	   + one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5])
	   + one_of_k_encoding(atom.GetTotalNumHs(), [0, 1, 2, 3, 4, 5])
	   + one_of_k_encoding(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5])
	   + [atom.IsInRing()]
	   + [atom.GetIsAromatic()]
	)
	#feature = np.array(feature, dtype=np.uint8)
	feature = np.packbits(feature)
	return feature


#Get features of an edge (one-hot encoding)
'''
	1.single/double/triple/aromatic: 4 dimensions       
	2.the atom's hybridization: 1 dimensions
	3.whether the bond is on ring: 1 dimension          
	Total: 6 dimensions
'''

def get_bond_feature(bond):
	bond_type = bond.GetBondType()
	feature = [
		bond_type == Chem.rdchem.BondType.SINGLE,
		bond_type == Chem.rdchem.BondType.DOUBLE,
		bond_type == Chem.rdchem.BondType.TRIPLE,
		bond_type == Chem.rdchem.BondType.AROMATIC,
		bond.GetIsConjugated(),
		bond.IsInRing()
	]
	#feature = np.array(feature, dtype=np.uint8)
	feature = np.packbits(feature)
	return feature


def smile_to_graph(smiles):
	mol = Chem.MolFromSmiles(smiles)
	N = mol.GetNumAtoms()
	node_feature = []
	edge_feature = []
	edge = []
	for i in range(mol.GetNumAtoms()):
		atom_i = mol.GetAtomWithIdx(i)
		atom_i_features = get_atom_feature(atom_i)
		node_feature.append(atom_i_features)

		for j in range(mol.GetNumAtoms()):
			bond_ij = mol.GetBondBetweenAtoms(i, j)
			if bond_ij is not None:
				edge.append([i, j])
				bond_features_ij = get_bond_feature(bond_ij)
				edge_feature.append(bond_features_ij)
	node_feature=np.stack(node_feature)
	edge_feature=np.stack(edge_feature)
	edge = np.array(edge,dtype=np.uint8)
	return N,edge,node_feature,edge_feature

def to_pyg_format(N,edge,node_feature,edge_feature):
	graph = Data(
		idx=-1,
		edge_index = torch.from_numpy(edge.T).int(),
		x          = torch.from_numpy(node_feature).byte(),
		edge_attr  = torch.from_numpy(edge_feature).byte(),
	)
	return graph

#debug one example
g = to_pyg_format(*smile_to_graph(smiles="C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1"))
print(g)
print('[Dy] is replaced by C !!')
print('smile_to_graph() ok!')

Data(x=[37, 9], edge_index=[2, 78], edge_attr=[78, 1], idx=-1)
[Dy] is replaced by C !!
smile_to_graph() ok!


In [154]:
INPUT_DIR = "../data/chunked-raw-dataset/"
input_path = INPUT_DIR + f"train_{0}.parquet"

n_rows = 10**5*3
train_raw = pl.read_parquet(input_path, n_rows=n_rows, columns=["molecule_smiles", "protein_name", "binds"]).to_pandas()
train = train_raw[train_raw['protein_name']=='BRD4'].copy()
train = train.drop(columns=['protein_name', "binds"]).reset_index(drop=True)
train['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
train['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
train['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values
binds = train[['bind1', 'bind2', 'bind3']]

In [134]:
#example of parallel conversion of smiles to graph

from multiprocessing import Pool
from tqdm import tqdm
import gc
from torch_geometric.loader import DataLoader as PyGDataLoader

def to_pyg_list(graph):
	L = len(graph)
	for i in tqdm(range(L)):
		N, edge, node_feature, edge_feature = graph[i]
		graph[i] = Data(
			idx=i,
			edge_index=torch.from_numpy(edge.T).int(),
			x=torch.from_numpy(node_feature).byte(),
			edge_attr=torch.from_numpy(edge_feature).byte(),
		)
	return graph


train_smiles=train['molecule_smiles'].values



num_train= len(train_smiles)
with Pool(processes=64) as pool:
    train_graph = list(tqdm(pool.imap(smile_to_graph, train_smiles), total=num_train))

train_graph = to_pyg_list(train_graph)



100%|██████████| 100000/100000 [00:04<00:00, 20870.14it/s]


In [135]:
import json
def save_data_to_parquet(data_list, bind_data, file_name):
    # DataオブジェクトをPandas DataFrameに変換
    data_dicts = []
    for data in data_list:
        data_dict = {
            'x': json.dumps(data.x.tolist()),
            'edge_index': json.dumps(data.edge_index.tolist()),
            'edge_attr': json.dumps(data.edge_attr.tolist()),
            'idx': data.idx
        }
        data_dicts.append(data_dict)
    graph_df = pd.DataFrame(data_dicts)
    
    # bindデータとグラフデータを結合
    combined_df = pd.concat([graph_df, bind_data], axis=1)
    
    # DataFrameをParquetファイルに保存
    combined_df.to_parquet(file_name, compression='snappy')

# データを読み込む関数
def load_data_from_parquet(file_name):
    # ParquetファイルからDataFrameを読み込み
    combined_df = pd.read_parquet(file_name)
    
    # DataFrameをDataオブジェクトに変換
    data_list = []
    for _, row in combined_df.iterrows():
        data = Data(
            x=torch.tensor(json.loads(row['x'])),
            edge_index=torch.tensor(json.loads(row['edge_index'])),
            edge_attr=torch.tensor(json.loads(row['edge_attr'])),
            idx=row['idx']
        )
        data_list.append(data)
    
    # bindデータを抽出
    bind_data = combined_df[['bind1', 'bind2', 'bind3']]
    
    return data_list, bind_data




In [155]:
# データを保存
data_list = train_graph
bind_data = binds
save_data_to_parquet(data_list, bind_data, 'graph_and_bind_data.parquet')

# データを読み込み
loaded_data_list, loaded_bind_data = load_data_from_parquet('graph_and_bind_data.parquet')

# # 読み込んだデータを確認
# for data in loaded_data_list:
#     print(data)


In [158]:
train_graph = loaded_data_list
train_bind = loaded_bind_data.to_numpy()

len(train_graph), train_bind.shape

(100000, (100000, 3))

In [159]:
train_bind

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [140]:
train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=False)


In [141]:
for batch in train_loader:
    print(batch.idx, batch.x.shape, batch.edge_index.shape, batch.edge_attr.shape)
    break

tensor([0, 1, 2]) torch.Size([118, 9]) torch.Size([2, 250]) torch.Size([250, 1])


In [180]:
#MODEL: simple MPNNModel
#from https://github.com/chaitjo/geometric-gnn-dojo/blob/main/geometric_gnn_101.ipynb

DEVICE='cpu'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# i have removed all comments here to jepp it clean. refer to orginal link for code comments
# of MPNNModel
class MPNNLayer(MessagePassing):
	def __init__(self, emb_dim=64, edge_dim=4, aggr='add'):
		super().__init__(aggr=aggr)

		self.emb_dim = emb_dim
		self.edge_dim = edge_dim
		self.mlp_msg = nn.Sequential(
			nn.Linear(2 * emb_dim + edge_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
			nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
		)
		self.mlp_upd = nn.Sequential(
			nn.Linear(2 * emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU(),
			nn.Linear(emb_dim, emb_dim), nn.BatchNorm1d(emb_dim), nn.ReLU()
		)

	def forward(self, h, edge_index, edge_attr):
		out = self.propagate(edge_index, h=h, edge_attr=edge_attr)
		return out

	def message(self, h_i, h_j, edge_attr):
		msg = torch.cat([h_i, h_j, edge_attr], dim=-1)
		return self.mlp_msg(msg)

	def aggregate(self, inputs, index):
		return scatter(inputs, index, dim=self.node_dim, reduce=self.aggr)

	def update(self, aggr_out, h):
		upd_out = torch.cat([h, aggr_out], dim=-1)
		return self.mlp_upd(upd_out)

	def __repr__(self) -> str:
		return (f'{self.__class__.__name__}(emb_dim={self.emb_dim}, aggr={self.aggr})')


class MPNNModel(nn.Module):
	def __init__(self, num_layers=4, emb_dim=64, in_dim=11, edge_dim=4, out_dim=1):
		super().__init__()

		self.lin_in = nn.Linear(in_dim, emb_dim)

		# Stack of MPNN layers
		self.convs = torch.nn.ModuleList()
		for layer in range(num_layers):
			self.convs.append(MPNNLayer(emb_dim, edge_dim, aggr='add'))

		self.pool = global_mean_pool

	def forward(self, data): #PyG.Data - batch of PyG graphs

		h = self.lin_in(F_unpackbits(data.x,-1).float())  

		for conv in self.convs:
			h = h + conv(h, data.edge_index.long(), F_unpackbits(data.edge_attr,-1).float())  # (n, d) -> (n, d)

		h_graph = self.pool(h, data.batch)  
		return h_graph

# our prediction model here !!!!
class Net(nn.Module):
	def __init__(self, ):
		super().__init__()

		self.output_type = ['infer', 'loss']

		graph_dim=96
		self.smile_encoder = MPNNModel(
			 in_dim=NODE_DIM, edge_dim=EDGE_DIM, emb_dim=graph_dim, num_layers=4,
		)
		self.bind = nn.Sequential(
			nn.Linear(graph_dim, 1024),
			#nn.BatchNorm1d(1024),
			nn.ReLU(inplace=True),
			nn.Dropout(0.1),
			nn.Linear(1024, 1024),
			#nn.BatchNorm1d(1024),
			nn.ReLU(inplace=True),
			nn.Dropout(0.1),
			nn.Linear(1024, 512),
			#nn.BatchNorm1d(512),
			nn.ReLU(inplace=True),
			nn.Dropout(0.1),
			nn.Linear(512, 3),
		)

	def forward(self, batch):
		graph = batch['graph']
		x = self.smile_encoder(graph) 
		bind = self.bind(x)

		# --------------------------
		output = {}
		if 'loss' in self.output_type:
			target = batch['bind']
			output['bce_loss'] = F.binary_cross_entropy_with_logits(bind.float(), target.float())
		if 'infer' in self.output_type:
			output['bind'] = torch.sigmoid(bind)

		return output
    
#debug: make some dummy data and run

def run_check_net():
	batch_size = 3
	node_dim=NODE_DIM
	edge_dim=EDGE_DIM

	data = []
	for b in range(batch_size):
		N = np.random.randint(5,10)
		E = np.random.randint(3,N*(N-1))
		edge_index = np.stack([
			np.random.choice(N, E, replace=True),
			np.random.choice(N, E, replace=True),
		]).T
		edge_index = np.sort(edge_index)
		edge_index = edge_index[edge_index[:, 0].argsort()]
		edge_index[0] = [0,1] #default
		edge_index = edge_index[edge_index[:,0]!=edge_index[:,1]]
		edge_index = np.unique(edge_index, axis=0)

		E = len(edge_index)
		edge_index = np.ascontiguousarray(edge_index.T)

		d = Data(
			idx        = b,
			edge_index = torch.from_numpy(edge_index).int(),
			x          = torch.from_numpy(np.packbits(np.random.choice(2, (N, node_dim)),-1)).byte(),
			edge_attr  = torch.from_numpy(np.packbits(np.random.choice(2, (E, edge_dim)),-1)).byte(),
		)
		data.append(d)

	#from my_mol2graph import make_dummy_data
	#data = make_dummy_data()

	loader = DataLoader(data, batch_size=batch_size)
	graph = next(iter(loader))
	idx = graph.idx.tolist()  #use to index bind array
	batch = dotdict( 
		graph = graph.to(DEVICE),
		bind  = torch.from_numpy(np.random.choice(2, (batch_size, 3))).float().to(DEVICE),
	)
	zz=0
 
	net = Net().to(DEVICE)
	#print(net)

	with torch.no_grad():
		with torch.cuda.amp.autocast(enabled=True): # dtype=torch.float16):
			output = net(batch)
			#print(output['bind'])

	# ---
	print('batch')
	for k, v in batch.items():
		if k=='idx':
			print(f'{k:>32} : {len(v)} ')
		elif k=='graph':
			print(f'{k:>32} : {graph} ')
		else:
			print(f'{k:>32} : {v.shape} ')

	print('output')
	for k, v in output.items():
		if 'loss' not in k:
			print(f'{k:>32} : {v.shape} ')
	print('loss')
	for k, v in output.items():
		if 'loss' in k:
			print(f'{k:>32} : {v.item()} ')

            
# run_check_net()
print('model ok!')

model ok!


In [75]:
# train_smiles = [ #replace [Dy] with C
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
#     "C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC#C)CC(=O)NC)n2)cc1",
# ]

# train_bind =np.array([
#     [0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,0,0],
# ])

# num_train= len(train_smiles)
# with Pool(processes=64) as pool:
#     train_graph = list(tqdm(pool.imap(smile_to_graph, train_smiles), total=num_train))

# train_graph = to_pyg_list(train_graph)
# train_loader = PyGDataLoader(train_graph, batch_size=3, shuffle=False)


100%|██████████| 6/6 [00:00<00:00, 7876.63it/s]


In [181]:
train_loader = PyGDataLoader(train_graph, batch_size=1024, shuffle=False)
NODE_DIM = 576
EDGE_DIM = 64

## example training loop
scaler = torch.cuda.amp.GradScaler(enabled=True)
net = Net()
net.to(DEVICE)

optimizer =\
	torch.optim.AdamW(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001)

num_epoch=10
epoch=0
iteration=0
while epoch<num_epoch: 
	for t, graph_batch in enumerate(train_loader): 
		index = graph_batch.idx.tolist()
		B = len(index)
		batch = dotdict(
			graph  = graph_batch.to(DEVICE),
			bind   = torch.from_numpy(train_bind[index]).to(DEVICE),
		)
		net.train()
		net.output_type = ['loss', 'infer']
		with torch.cuda.amp.autocast(enabled=True):
			output = net(batch)  #data_parallel(net,batch) #
			bce_loss = output['bce_loss']

		optimizer.zero_grad() 
		scaler.scale(bce_loss).backward() 
		scaler.step(optimizer)
		scaler.update()
		 
		torch.clear_autocast_cache()
		print(epoch,iteration,bce_loss.item())
		iteration +=  1
        
	epoch += 1

0 0 0.6924242973327637
0 1 0.30133605003356934
0 2 0.03894919902086258
0 3 0.020042411983013153
0 4 0.019090522080659866
0 5 0.021832814440131187
0 6 0.0136260986328125
0 7 0.4060516357421875
0 8 0.0412495955824852
0 9 0.095245361328125
0 10 0.04803466796875
0 11 0.0528920516371727
0 12 0.028544317930936813
0 13 0.06625741720199585
0 14 0.1799229234457016
0 15 0.03147873282432556
0 16 0.016117390245199203
0 17 0.028878819197416306
0 18 0.01669585146009922
0 19 0.03128521889448166
0 20 0.008573664352297783
0 21 0.007935790345072746
0 22 0.018975595012307167
0 23 0.015594346448779106
0 24 0.021374858915805817
0 25 0.009400422684848309
0 26 0.012025441974401474
0 27 0.018769454210996628
0 28 0.00711430236697197
0 29 0.003959357272833586
0 30 0.010211670771241188
0 31 0.027804581448435783
0 32 0.0013063312508165836
0 33 0.005339504685252905
0 34 0.013329213485121727
0 35 0.012350231409072876
0 36 0.007432190235704184
0 37 0.02281390130519867
0 38 0.005795703269541264
0 39 0.005676069296896

In [182]:
# inference
net.eval()
net.output_type = ['infer']
preds = []
with torch.no_grad():
    for t, graph_batch in enumerate(train_loader): 
        index = graph_batch.idx.tolist()
        B = len(index)
        batch = dotdict(
            graph  = graph_batch.to(DEVICE),
            bind   = torch.from_numpy(train_bind[index]).to(DEVICE),
        )
        output = net(batch)
        preds.append(output['bind'].cpu().numpy())
preds = np.concatenate(preds)

preds 

array([[1.32094498e-03, 1.27357699e-03, 1.50462403e-03],
       [1.60203874e-03, 1.18648622e-03, 6.29884657e-04],
       [6.07801310e-04, 6.84889616e-04, 2.16376595e-03],
       ...,
       [3.22368760e-06, 2.27618034e-06, 1.10506555e-08],
       [8.07896868e-06, 5.70151542e-06, 5.79282577e-08],
       [2.79794385e-05, 2.26809807e-05, 1.01579623e-07]], dtype=float32)

In [183]:
from sklearn.metrics import average_precision_score as APS
APS(train_bind, preds)


0.12299345104476433